In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [6]:
import logging
import parkit.api as pk
logger = logging.getLogger(__name__)

In [7]:
pk.set_default_site('C:\\Users\\rdpuser\\Desktop\\test', create = True)

In [8]:
# Collection Tests

In [9]:
pk.get_default_site()

('C:\\Users\\rdpuser\\Desktop\\test', '3da7cdfc-aca9-468b-a8c6-e1bc2b1c92d1')

In [10]:
pk.bind_symbols('module')

['collection_tests']

In [12]:
collection_tests(1000, sync = True)

True

In [13]:
pk.disable_tasks()
pk.compactify()
assert len([e for e in pk.task_executions()]) == 0

In [14]:
list(pk.task_executions())

[]

In [15]:
pk.enable_tasks()

In [16]:
pk.set_concurrency(4)

In [17]:
import random

executions = []
for i in range(16):
    executions.append(collection_tests(1000))
while True:
    assert len(executions) == 16
    if all(e.status in ['crashed', 'failed', 'finished'] for e in executions):
        break
    choice = random.choice([1, 2])
    if choice == 1:
        pk.set_concurrency(random.randint(1, 10))
    elif choice == 2:
        pk.enable_tasks()
assert all(e.result for e in executions)
assert all(e.status == 'finished' for e in executions)
assert all(e.error is None for e in executions)

In [18]:
[e.status for e in executions]

['finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished']

In [19]:
pk.get_concurrency()

10

In [20]:
pk.disable_tasks()

In [21]:
[e.status for e in pk.task_executions()]

['finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished',
 'finished']

In [58]:
# Scheduler Tests

In [52]:
@pk.task
def foo():
    logger.info('foo')

In [56]:
pk.enable_tasks()

In [55]:
pk.schedule(foo, frequency = pk.Frequency.Second, period = 1, max_times = 10, start = 'now')

In [57]:
[s.cancel() for s in pk.schedulers()]

[None]

In [59]:
pk.compactify()

In [ ]:
# File Tests

In [23]:
f = pk.File()

In [24]:
f.set_content('hello')

In [25]:
assert len(f.get_content()) == 5

In [26]:
assert f.get_content() == 'hello'

In [27]:
assert f.encoding == 'utf-8'

In [28]:
assert f.metadata['content-type'] == 'text/plain'

In [29]:
f.set_content(b'hello')

In [30]:
assert f.get_content() == b'hello'

In [31]:
assert isinstance(f.get_content(), bytearray)

In [32]:
with pk.snapshot(f):
    assert isinstance(f.get_content(zero_copy = True), memoryview)

In [33]:
assert f.metadata['content-type'] == 'application/octet-stream'

In [34]:
f.set_content([1, 2, 3])

In [35]:
assert f.get_content() == [1, 2, 3]

In [36]:
assert f.metadata['content-type'] == 'application/python-pickle'

In [37]:
import numpy as np
import pandas as pd

In [38]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [39]:
f = pk.File(mode = 'wb')

In [40]:
assert f.size == 0

In [41]:
f.set_content(df)

In [42]:
assert df.equals(f.get_content())

In [43]:
a = np.random.rand(100,100)

In [45]:
f = pk.File('test', mode = 'wb+')
with f:
    np.save(f, a)

In [46]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))

In [48]:
f = pk.File('test', mode = 'w+')
with f:
    np.savetxt(f, a)

In [49]:
f.mode = 'r+'
with f:
    assert np.array_equal(a, np.loadtxt(f))

In [50]:
for obj in pk.directory('__exec__', include_hidden = True):
    print(obj.path)

__exec__/__cluster_state_dict__
__exec__/__node_termination_queue__
__exec__/__running_dict__
__exec__/__submit_queue__
